<a href="https://colab.research.google.com/github/MunYeong0723/gj-AI_WarmingUp/blob/MunYeong/Training_With_YoloV4_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s "/content/drive/My Drive/" "/myDrive"
!ls /myDrive

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# # use this to upload files
# def upload():
#   from google.colab import files
#   uploaded = files.upload() 
#   for name, data in uploaded.items():
#     with open(name, 'wb') as f:
#       f.write(data)
#       print ('saved file', name)

# # use this to download a file  
# def download(path):
#   from google.colab import files
#   files.download(path)

In [ ]:
!ls /myDrive/yolov4

In [ ]:
# download cfg to google drive and change its name
!cp cfg/yolov4-tiny-custom.cfg /myDrive/yolov4/yolov4-tiny-obj.cfg

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
# !cp "/content/drive/My Drive/Colab Notebooks/yolov4/yolov4-obj.cfg" ./cfg

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
# !cp "/content/drive/My Drive/Colab Notebooks/yolov4/obj.names" ./data
# !cp "/content/drive/My Drive/Colab Notebooks/yolov4/obj.data"  ./data

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
# !cp "/content/drive/My Drive/Colab Notebooks/yolov4/generate_train.py" ./
# !cp "/content/drive/My Drive/Colab Notebooks/yolov4/generate_test.py" ./

In [ ]:
!python /myDrive/yolov4/generate_train.py
!python /myDrive/yolov4/generate_valid.py

# Download pre-trained weights for the convolutional layers

In [ ]:
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False

# Edit .cfg

I recommend having **batch = 64** and **subdivisions = 16** for ultimate results. If you run into any issues then up subdivisions to 32.

Make the rest of the changes to the cfg based on how many classes you are training your detector on.

**Note:** 
I set my **max_batches = 6000**, **steps = 4800, 5400**, I changed the **classes = 1** in the three YOLO layers and **filters = 18** in the three convolutional layers before the YOLO layers.

How to Configure Your Variables:

width = 416

height = 416
**(these can be any multiple of 32, 416 is standard, you can sometimes improve results by making value larger like 608 but will slow down training)**

max_batches = (# of classes) * 2000
**(but no less than 6000 so if you are training for 1, 2, or 3 classes it will be 6000, however detector for 5 classes would have max_batches=10000)**

steps = (80% of max_batches), (90% of max_batches)
**(so if your max_batches = 10000, then steps = 8000, 9000)**

filters = (# of classes + 5) * 3
**(so if you are training for one class then your filters = 18, but if you are training for 4 classes then your filters = 27)**


**Optional:** If you run into memory issues or find the training taking a super long time. In each of the three yolo layers in the cfg, change one line from random = 1 to **random = 0** to speed up training but slightly reduce accuracy of model. Will also help save memory if you run into any memory issues.


# Train Your Custom Object Detector

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
# !./darknet detector train /myDrive/yolov4/obj.data /myDrive/yolov4/yolov4-obj.cfg yolov4.conv.137 -dont_show -map
!./darknet detector train /myDrive/yolov4/obj.data /myDrive/yolov4/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

In [ ]:
# show chart.png of how custom object detector did with training
imShow('chart.png')